In [17]:
"""Specification Test:  Non-nested hypotheses: Composite model.

Swiss Metro Case Study
Demand Modeling: 1.202
Spring 2019
Alexandra Berke

"""

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

pandas = pd.read_table("../Data/swissmetro.dat")
database = db.Database("swissmetro",pandas)

from headers import *

# Specify parameters to estimate
#1: the name of the parameter,
#2: the default value,
#3: a lower bound (or None, if no bound is specified),
#4: an upper bound, (or None, if no bound is specified),
#5: a flag that indicates if the parameter must be estimated (0) or if it keeps its default value.
ASC_CAR	 = Beta('ASC_CAR',0,-20,20,0)
ASC_SBB	 = Beta('ASC_SBB',0,-10,10,1)
ASC_SM	 = Beta('ASC_SM',0,-20,20,0)
B_CAR_COST	 = Beta('B_CAR_COST',0,-10,10,0)
B_HE	 = Beta('B_HE',0,-10,10,0)
B_SM_COST	 = Beta('B_SM_COST',0,-10,10,0)
B_CAR_TIME	 = Beta('B_CAR_TIME',0,-10,10,0)
B_TRAIN_TIME	 = Beta('B_TRAIN_TIME',0,-10,10,0)
B_SM_TIME	 = Beta('B_SM_TIME',0,-10,10,0)
B_TRAIN_COST	 = Beta('B_TRAIN_COST',0,-10,10,0)
B_SENIOR	 = Beta('B_SENIOR',0,-10,10,0)
B_GA	 = Beta('B_GA',0,-10,80,0)
B_FIRST	 = Beta('B_FIRST',0,-10,80,0)
B_SM_MALE = Beta('B_SM_MALE',0,-10,10,0)
B_CAR_MALE = Beta('B_CAR_MALE',0,-10,10,0)
B_TRAIN_MALE = Beta('B_TRAIN_MALE',0,-10,10,0)
B_SM_LOW_INCOME = Beta('B_SM_LOW_INCOME',0,-10,10,0)
B_SM_HIGH_INCOME	 = Beta('B_SM_HIGH_INCOME',0,-10,10,0)
B_CAR_LOW_INCOME = Beta('B_CAR_LOW_INCOME',0,-10,10,0)
B_CAR_HIGH_INCOME	 = Beta('B_CAR_HIGH_INCOME',0,-10,10,0)
B_TRAIN_LOW_INCOME = Beta('B_TRAIN_LOW_INCOME',0,-10,10,0)
B_TRAIN_HIGH_INCOME	 = Beta('B_TRAIN_HIGH_INCOME',0,-10,10,0)


# Define here arithmetic expressions for name that are not directly 
# available from the data

SENIOR  = DefineVariable('SENIOR', AGE   ==  5,database )
LOW_INCOME  = DefineVariable('LOW_INCOME', INCOME  ==  1,database )
HIGH_INCOME  = DefineVariable('HIGH_INCOME', INCOME  ==  3,database )
CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST  = DefineVariable('SM_COST', SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
one  = DefineVariable('one',1,database)

#Utilities
Car_SP = ASC_CAR * one + B_CAR_TIME * CAR_TT + B_CAR_COST * CAR_CO + B_SENIOR * SENIOR \
        + B_CAR_LOW_INCOME * LOW_INCOME + B_CAR_HIGH_INCOME * HIGH_INCOME + B_CAR_MALE * MALE
SBB_SP = ASC_SBB * one + B_TRAIN_TIME * TRAIN_TT + B_TRAIN_COST * TRAIN_COST + B_HE * TRAIN_HE \
        + B_GA * GA + B_FIRST * FIRST \
        + B_TRAIN_LOW_INCOME * LOW_INCOME + B_TRAIN_HIGH_INCOME * HIGH_INCOME + B_TRAIN_MALE * MALE
SM_SP = ASC_SM * one + B_SM_TIME * SM_TT + B_SM_COST * SM_COST + B_HE * SM_HE + B_GA * GA \
        + B_SENIOR * SENIOR + B_FIRST * FIRST \
        + B_SM_LOW_INCOME * LOW_INCOME + B_SM_HIGH_INCOME * HIGH_INCOME + B_SM_MALE * MALE

V = {3: Car_SP,1: SBB_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Here we use the "biogeme" way for backward compatibility
exclude = (((  PURPOSE   !=  1  ) * (  PURPOSE   !=  3  )) + (  CHOICE   ==  0  ) + (  AGE == 6  ))>0
database.remove(exclude)

# Estimation of the model
logprob = bioLogLogit(V,av,CHOICE)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "SpecTest_SM_MC"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_CAR=	-1.01
ASC_SM=	-0.237
B_CAR_COST=	-0.00795
B_CAR_HIGH_INCOME=	0.149
B_CAR_LOW_INCOME=	0.196
B_CAR_MALE=	0.593
B_CAR_TIME=	-0.0131
B_FIRST=	0.295
B_GA=	0.681
B_HE=	-0.00648
B_SENIOR=	-1.73
B_SM_COST=	-0.0118
B_SM_HIGH_INCOME=	0.186
B_SM_LOW_INCOME=	-0.324
B_SM_MALE=	0.0869
B_SM_TIME=	-0.0109
B_TRAIN_COST=	-0.0265
B_TRAIN_HIGH_INCOME=	-0.335
B_TRAIN_LOW_INCOME=	0.128
B_TRAIN_MALE=	-0.68
B_TRAIN_TIME=	-0.00825
                        Value   Std err     t-test       p-value  \
ASC_CAR             -1.007639  0.179597  -5.610541  2.016951e-08   
ASC_SM              -0.237179  0.138094  -1.717524  8.588356e-02   
B_CAR_COST          -0.007945  0.001081  -7.351935  1.953993e-13   
B_CAR_HIGH_INCOME    0.149183  0.055334   2.696056  7.016586e-03   
B_CAR_LOW_INCOME     0.195736  0.095035   2.059613  3.943552e-02   
B_CAR_MALE           0.593008  0.070446   8.417848  0.000000e+00   
B_CAR_TIME          -0.013053  0.000815 -16.016447  0.000000e+00   
B_FIRST              0.294762  0.0785